# Predicción del consumo de agua - Driscoll's 2025

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# 1. Cargar base simulada
df = pd.read_csv("base_fake.csv")
df["Fecha"] = pd.to_datetime(df["Fecha"])

# 2. Variables base
df["m3_usados"] = df.groupby(["Productor", "Cultivo"])["Lectura m3"].diff().fillna(0)
df["Litros_usados"] = (df["m3_usados"] * 1000 / df["Hectareas"]).clip(lower=0)

# 3. Entrenamiento modelo
X = pd.get_dummies(df[["Semana", "Mes", "Año", "Cultivo", "Zona de Produccion"]], drop_first=True)
y = df["Litros_usados"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 4. Simular predicciones 2025
productores_unicos = df[["Productor", "Cultivo", "Zona de Produccion"]].drop_duplicates()
future_weeks = pd.date_range(start="2025-01-01", periods=50, freq="W")
pred_rows = []
for _, row in productores_unicos.iterrows():
    for i, fecha in enumerate(future_weeks, start=1):
        pred_rows.append({
            "Productor": row["Productor"],
            "Cultivo": row["Cultivo"],
            "Zona de Produccion": row["Zona de Produccion"],
            "Semana": i,
            "Mes": fecha.month,
            "Año": fecha.year,
            "Fecha": fecha
        })
df_pred = pd.DataFrame(pred_rows)

# 5. Predicción
df_pred_encoded = pd.get_dummies(df_pred[["Semana", "Mes", "Año", "Cultivo", "Zona de Produccion"]], drop_first=True)
df_pred_encoded = df_pred_encoded.reindex(columns=X.columns, fill_value=0)
df_pred["Litros_estimado"] = model.predict(df_pred_encoded)

# 6. Simulación de clima
np.random.seed(42)
df_clima = pd.DataFrame({
    "Fecha": pd.date_range(start="2025-01-01", end="2025-12-31", freq="D"),
    "Temperatura": np.random.normal(18, 3, 365),
    "Precipitacion": np.random.exponential(0.5, 365)
})
df_clima["Semana"] = df_clima["Fecha"].dt.isocalendar().week
df_clima["Año"] = df_clima["Fecha"].dt.year
df_clima_semanal = df_clima.groupby(["Año", "Semana"]).agg({"Temperatura": "mean", "Precipitacion": "sum"}).reset_index()
df_clima_semanal["Necesidad_Litros_Ha_semana"] = (50 + df_clima_semanal["Temperatura"] * 10 - df_clima_semanal["Precipitacion"] * 5).clip(lower=50, upper=300)

# 7. Unir predicción con clima
df_final = df_pred.merge(df_clima_semanal, on=["Año", "Semana"], how="left")
df_final["Diferencia_vs_clima"] = df_final["Litros_estimado"] - df_final["Necesidad_Litros_Ha_semana"]

# 8. Clasificación eficiencia
df_final["Rendimiento_vs_clima"] = df_final["Diferencia_vs_clima"].apply(lambda d: "Sobreconsumo" if d > 30 else "Ahorro" if d < -30 else "Adecuado")

# 9. Mapa
zona_coords = {
    "Tuxcueca": (20.2, -103.4),
    "Guzmán": (19.7, -103.5),
    "Zamora": (19.98, -102.28),
    "Jocotepec": (20.28, -103.43),
    "Tangancícuaro": (19.88, -102.22)
}
map_data = df_final.groupby(["Productor", "Zona de Produccion"])[["Litros_estimado", "Diferencia_vs_clima"]].mean().reset_index()
m = folium.Map(location=[20.2, -103.3], zoom_start=7)
for _, row in map_data.iterrows():
    lat, lon = zona_coords.get(row["Zona de Produccion"], (20.0, -103.0))
    color = "green" if row["Diferencia_vs_clima"] < -30 else "red" if row["Diferencia_vs_clima"] > 30 else "blue"
    folium.CircleMarker(
        location=(lat, lon),
        radius=8,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['Productor']} ({row['Zona de Produccion']})<br>Consumo: {row['Litros_estimado']:.1f} L/Ha<br>Diferencia: {row['Diferencia_vs_clima']:.1f}"
    ).add_to(m)
m.save("mapa_productores_consumo_2025.html")
print("✅ Mapa guardado como HTML")

KeyError: 'Column not found: Lectura m3'